# **IAA - PRÀCTICA: MAIN**

### **Instal·lar llibreries necessàries**

In [ ]:
%pip install -r ../assets/requirements.txt 

### **Importar llibreries**

In [1]:
def import_dependencies():
	global pd, np, plt, sns, skl

	import pandas as pd
	import numpy as np
	import matplotlib.pyplot as plt
	import seaborn as sns
	import sklearn as skl

import_dependencies()

### **Llegir les dades (Cirrhosis Dataset)**

In [2]:
def load_dataset(save_to_csv: bool = True):
	global data
	from ucimlrepo import fetch_ucirepo 
	
	# Fetch dataset
	cirrhosis_patient_survival_prediction = fetch_ucirepo(id=878)

	data = pd.DataFrame(cirrhosis_patient_survival_prediction.data.original)

	if save_to_csv:
		# Guardem el dataset per poder-lo visualitzar sencer
		data.to_csv('../assets/data/raw_cirrhosis.csv', index=False)

#load_dataset(save_to_csv=True)

### **Informació del dataset inicial**

In [ ]:
data.shape

In [ ]:
data.head(-10)

In [ ]:
data.info()

### **Preprocessing inicial**

In [3]:
def initial_preprocessing(data: pd.DataFrame, save_to_csv: bool = True):
	"""
	Reemplaça els valors 'NaNN' per NaN, assigna els tipus de dades correctes a cada columna i renombra les classes d'algunes variables per una millor comprensió.
	"""
	# Reemplaçar l'string 'NaNN' per NaN
	data.replace(to_replace=['NaNN', '', pd.NA], value=np.nan, inplace=True)

	# Eliminem variables que no aporten informació
	data.drop(columns=['ID'], inplace=True)

	# Assignem els tipus de dades correctes a cada columna
	int64_variables = ['N_Days', 'Age', 'Cholesterol', 'Copper', 'Tryglicerides', 'Platelets']
	float64_variables = ['Bilirubin', 'Albumin', 'Alk_Phos', 'SGOT', 'Prothrombin']
	category_variables = ['Status', 'Drug', 'Sex', 'Ascites', 'Hepatomegaly', 'Spiders', 'Edema', 'Stage']
	boolean_variables = ['Ascites', 'Hepatomegaly', 'Spiders']

	data[int64_variables] = data[int64_variables].astype('Int64')
	data[float64_variables] = data[float64_variables].astype('float64')
	data[category_variables] = data[category_variables].astype('category')

	global original_column_types

	original_column_types = {col: str(data[col].dtype) for col in data.columns} # Guardem els tipus de dades de cada columna

	# Renombrem les classes d'algunes variables per una millor comprensió
	data['Status'] = data['Status'].replace({'D': 'Dead', 'C': 'Alive', 'CL': 'LiverTransplant'})
	data[boolean_variables] = data[boolean_variables].replace({'Y': 1, 'N': 0})
	data['Edema'] = data['Edema'].replace({'N': 'NoEdema', 'S': 'EdemaResolved', 'Y': 'EdemaPersistent'})

	global original_categorical_categories
	original_categorical_categories = {col: data[col].unique() for col in data.select_dtypes(include='category').columns} # Guardem les categories originals de les variables categòriques

	if save_to_csv:
		# Guardem el dataset
		data.to_csv('../assets/data/initial_preprocessing_cirrhosis.csv', index=False)

#initial_preprocessing(data=data, save_to_csv=True)

In [ ]:
data.head(-10)

In [ ]:
data.info()

### **Anàlisis inicial de les variables**

In [ ]:
data.head(-10)

In [ ]:
data.isna().sum().sort_values(ascending=False)

In [ ]:
# Estudi de les variables numèriques
data.describe()

In [ ]:
# Estadístiques de les variables categòriques
data.describe(include='category')

In [ ]:
def numerical_vars_histograms(data: pd.DataFrame):
    # Visualització de les distribucions de les variables numèriques en una sola figura
    numerical_columns = data.select_dtypes(include=['Int64', 'float64']).columns

    num_rows = int(np.ceil(len(numerical_columns) / 2))

    fig = plt.figure(figsize=(10, num_rows * 4))

    for i, col in enumerate(numerical_columns):
        ax = fig.add_subplot(num_rows, 2, i + 1)
        
        sns.histplot(data[col], edgecolor="k", linewidth=1.5, kde=True)
        
        plt.xticks(rotation=45, ha='right')
        
        ax.set_title(f'Distribució de la variable numèrica {col}')
        ax.set_xlabel(col)
        ax.set_ylabel('Freqüència')

    plt.tight_layout()
    plt.show()

#numerical_vars_histograms(data=data)

In [ ]:
def categorical_vars_countplots(data: pd.DataFrame):
    """
    Visualització de les distribucions de les variables categòriques en una sola figura.
    """
    # Visualització de les distribucions de les variables categòriques en una sola figura
    categorical_columns = data.select_dtypes(include=['category']).columns
    num_rows = int(np.ceil(len(categorical_columns) / 2))

    fig = plt.figure(figsize=(10, num_rows * 4))

    for i, col in enumerate(categorical_columns):
        ax = fig.add_subplot(num_rows, 2, i + 1)
        
        sns.countplot(data=data, x=col, ax=ax, hue=col, legend=False)
        
        plt.xticks(rotation=45, ha='right')
        
        ax.set_title(f'Distribució de la variable categòrica {col}')
        ax.set_xlabel(col)
        ax.set_ylabel('Quantitat')

    plt.tight_layout()
    plt.show()

#categorical_vars_countplots(data=data)

### **Tractament d'outliers**

In [ ]:
def compare_iqr_factors(data: pd.DataFrame, factors: list = [1.5, 1.75, 2, 2.25, 2.5, 2.75, 3, 3.25, 3.5, 3.75, 4, 4.25, 4.5, 4.75, 5]):
	"""
	Compara diferents factors que multipliquen al IQR per a determinar els outliers i realitza un gràfic evolutiu per comparar-los.
	"""
	numerical_columns = data.select_dtypes(include=['Int64', 'float64']).columns

	plt.figure(figsize=(10, 6))

	# Dictionary to store outlier percentages for each factor and column
	outlier_percentages = {col: [] for col in numerical_columns}
	total_percentages = [set() for _ in range(len(factors))]

	for col in numerical_columns:
		Q1 = data[col].quantile(0.25)
		Q3 = data[col].quantile(0.75)
		IQR = Q3 - Q1

		for f_id, factor in enumerate(factors):
			outliers_mask = ((data[col] < (Q1 - factor * IQR)) | (data[col] > (Q3 + factor * IQR)))
			total_percentages[f_id].update(data.index[outliers_mask])
			outliers_percentage = np.mean(outliers_mask) * 100
			outlier_percentages[col].append(outliers_percentage)

	total_percentages = [(len(outliers) / len(data)) * 100 for outliers in total_percentages]
			
	# Plotting the results
	for col, percentages in outlier_percentages.items():
		plt.plot(factors, percentages, label=col)
	plt.plot(factors, total_percentages, label='Total', linestyle='--', color='black')

	plt.xlabel('Factor multiplicatiu del IQR')
	plt.ylabel('Percentage d\'outliers (%)')
	plt.title('Percentatge d\'outliers de cada variable numèrica per a diferents factors multiplicatius del IQR')
	plt.xticks(factors)
	
	plt.legend()
	plt.grid(True)
	plt.show()

#compare_iqr_factors(data=data, factors=[1.5, 1.75, 2, 2.25, 2.5, 2.75, 3, 3.25, 3.5, 3.75, 4, 4.25, 4.5, 4.75, 5])

In [4]:
def delete_outliers(data: pd.DataFrame, factor: float = 1.5, plots: bool = True, save_to_csv: bool = True):
    """
    Funció que detecta, visualitza i elimina els outliers d'un dataset. El factor multiplica el IQR per a determinar quins valors són outliers.
    """
    # Detecció, visualització i eliminació d'outliers
    numerical_columns = data.select_dtypes(include=['Int64', 'float64']).columns

    outliers_indices = []

    for col in numerical_columns:
        Q1 = data[col].quantile(0.25)
        Q3 = data[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - factor * IQR
        upper_bound = Q3 + factor * IQR
        outliers_mask = ((data[col] < lower_bound) | (data[col] > upper_bound))
        outliers = data[col][outliers_mask]
        non_outliers = data[col][~outliers_mask]

        outliers_indices.extend(data[col][outliers_mask].index.tolist())
        
        if plots:
            fig, axes = plt.subplots(2, 2, figsize=(8, 6))

            # Boxplot con los outliers originales
            sns.boxplot(ax=axes[0, 0], y=data[col], orient='v')
            axes[0, 0].scatter(x=[0]*len(outliers), y=outliers, color='red', marker='o')
            axes[0, 0].set_title(f'Boxplot de {col} con outliers ({factor}x IQR)')

            # Histograma con línea vertical para outliers
            sns.histplot(ax=axes[0, 1], data=data, x=col, kde=True)
            if (data[col] < lower_bound).any():
                axes[0, 1].axvline(x=lower_bound, color='red', linestyle='dashed')
            if (data[col] > upper_bound).any():
                axes[0, 1].axvline(x=upper_bound, color='red', linestyle='dashed')
            axes[0, 1].set_title(f'Histograma de {col}')
            axes[0, 1].set_xlabel(col)
            axes[0, 1].set_ylabel('Frecuencia')

            # Boxplot sin los outliers
            sns.boxplot(ax=axes[1, 0], y=non_outliers, orient='v')
            axes[1, 0].set_title(f'Boxplot de {col} sin outliers')

            # Histograma sin los outliers
            sns.histplot(ax=axes[1, 1], data=data[~outliers_mask], x=col, kde=True)
            axes[1, 1].set_title(f'Histograma de {col} sin outliers')

            percent_outliers = len(outliers) / data.shape[0] * 100
            fig.text(x=0.5, y=0, s=f'Outliers de {col} ({factor}x IQR): {len(outliers)} ({percent_outliers:.2f}%)', 
                    ha='center', va='center')

            plt.tight_layout()
            plt.show()


    unique_outliers = len(set(outliers_indices))

    print(f"Datset amb outliers: {data.shape[0]} files i {data.shape[1]} columnes.")
    print(f"Nombre total d'outliers únics eliminats: {unique_outliers} ({unique_outliers / data.shape[0] * 100:.2f}% de tot el dataset).")

    # Eliminació d'outliers
    data.drop(list(set(outliers_indices)), inplace=True)

    
    print(f"Dataset sense outliers: {data.shape[0]} files i {data.shape[1]} columnes.")

    if save_to_csv:
        # Guardem el dataset
        data.to_csv('../assets/data/no_outliers_cirrhosis.csv', index=False)

#delete_outliers(data=data, factor=3, plots=False, save_to_csv=True)

### **Funció per tractar les últimes files del dataset (amb molts missings)**

In [5]:
def delete_last_rows(data: pd.DataFrame, save_to_csv: bool = True):
	"""
	Elimina les últimes files del dataset (les que contenen 9 NaNs).
	"""
	# Eliminem les últimes files del dataset (les que contenen 9 NaNs)
	data.drop(data[data.isna().sum(axis=1) == 9].index, inplace=True)

	if save_to_csv:
		# Guardem el dataset
		data.to_csv('../assets/data/last_rows_deleted_cirrhosis.csv', index=False)

#delete_last_rows(data=data, save_to_csv=True)

### **Recodificació de variables categòriques**

In [24]:
def encode_variables(data: pd.DataFrame, save_to_csv: bool = True):
    """
    Codifica les variables categòriques que calgui per a poder-les utilitzar en els models de ML. 
    A més, guarda el mapping per a poder decodificar-les.
    """
    assert not data.isna().any().any(), "Per codificar el dataset cal que no hi hagi valors NaN."

    from sklearn.preprocessing import OneHotEncoder

    global ohe_mapping, original_columns_order

    original_columns_order = data.columns

    columns_to_encode = ['Drug', 'Sex', 'Edema', 'Stage'] # Sense la variable 'Status' perquè és la target i, a més, no té valors NaN

    new_encoded_columns_per_old_encoded_column = {col: set() for col in columns_to_encode} # Guardem les classes de cada columna a codificar

    ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

    data_encoded = ohe.fit_transform(data[columns_to_encode])
    encoded_columns = ohe.get_feature_names_out(columns_to_encode)

    # Guardem el mapping per a poder decodificar les variables
    ohe_mapping = {}
    for i, col in enumerate(columns_to_encode):
        # for category in original_categorical_categories[col]:
        #     new_encoded_column_name = f"{col}_{category}"
        #     ohe_mapping[new_encoded_column_name] = (col, category)
        #     new_encoded_columns_per_old_encoded_column[col].add(new_encoded_column_name)

        for category in ohe.categories_[i]:
            new_encoded_column_name = f"{col}_{category}"
            ohe_mapping[new_encoded_column_name] = (col, category)
            new_encoded_columns_per_old_encoded_column[col].add(new_encoded_column_name)

    data[encoded_columns] = data_encoded
    data[encoded_columns] = data[encoded_columns].astype('category')

    # Eliminem les columnes originals
    data.drop(columns=columns_to_encode, inplace=True)

    if save_to_csv:
        # Guardem el dataset
        data.to_csv('../assets/data/encoded_cirrhosis.csv', index=False)

#encode_variables(data=data, save_to_csv=True)

In [26]:
def decode_variables(data: pd.DataFrame, ohe_mapping, original_columns_order):
    """
    Decodifica les variables categòriques que s'hagin codificat anteriorment.
    """
    reconstructed_columns = {}

    # Creem les columnes reconstruïdes
    for encoded_column in ohe_mapping:
        if encoded_column in data.columns:
            original_column, category = ohe_mapping[encoded_column]

            if original_column not in reconstructed_columns:
                reconstructed_columns[original_column] = pd.Series([np.nan] * len(data), index=data.index, dtype='object')

            category_rows = data[encoded_column] == 1
            reconstructed_columns[original_column].loc[category_rows] = category

    # Eliminem les columnes codificades
    data.drop(columns=[col for col in ohe_mapping if col in data.columns], inplace=True)

    # Inserim les columnes reconstruïdes al DataFrame
    for col in reconstructed_columns:
        data[col] = reconstructed_columns[col]
        data[col] = data[col].astype(original_column_types[col])

    # Reordenem les columnes perquè quedin igual que a l'original
    data = data.reindex(columns=original_columns_order)

#decode_variables(data=data, ohe_mapping=ohe_mapping, original_columns_order=original_columns_order)

### **Partició del dataset en train/test**

In [8]:
def split_dataset(data: pd.DataFrame, test_size: float = 0.15, stratify: bool = True, random_state: int = 42):
	"""
	Particiona el dataset en conjunts de entrenament i prova.
	"""
	global train, test, X_train, y_train, X_test, y_test
	
	from sklearn.model_selection import train_test_split

	# Particionem el dataset en conjunts de entrenament i prova
	if stratify:
		train, test = train_test_split(data, test_size=test_size, random_state=random_state, stratify=data['Status'])
	else:
		train, test = train_test_split(data, test_size=test_size, random_state=random_state)

	# 'Status' es la variable target
	X_train = train.drop(columns=['Status'])
	y_train = train['Status']
	X_test = test.drop(columns=['Status'])
	y_test = test['Status']

	print(f"Train shape: {train.shape}")
	print(f"Test shape: {test.shape}")

#split_dataset(data=data, test_size=0.15, random_state=42)

### **Escalar variables numèriques**

In [9]:
def scale_variables(data: pd.DataFrame, scaler: str = 'standard', save_to_csv: bool = True):
	assert scaler in ['standard', 'minmax'], "El paràmetre 'scaler' ha de ser 'standard' o 'minmax'."
	"""
	Escala les variables numèriques.
	"""
	from sklearn.preprocessing import StandardScaler, MinMaxScaler

	numerical_columns = data.select_dtypes(include=['Int64', 'float64']).columns

	sc = StandardScaler() if scaler == 'standard' else MinMaxScaler()

	data[numerical_columns] = sc.fit_transform(data[numerical_columns])

	if save_to_csv:
		# Guardem el dataset
		data.to_csv('../assets/data/scaled_cirrhosis.csv', index=False)

# scale_variables(data=data, scaler='standard', save_to_csv=True)
# scale_variables(data=data, scaler='minmax', save_to_csv=True)

### **Imputar els valors faltants (Missings)**

In [ ]:
# Imprimeix les variables que tenen valors NaN, el seu percentatge i el seu tipus de dades
for col_train in train.columns:
	if train[col_train].isna().any():
		print(f"{col_train}: {train[col_train].isna().sum()} NaNs ({train[col_train].isna().sum() / len(train) * 100:.2f}%) ({train[col_train].dtype})")

In [18]:
def find_best_imputer(
                X_train: pd.DataFrame, \
                folds_cross_val: int = 5, \
                random_state: int = 42, \
                print_scores: bool = True, \
                name_num_metric: str = 'r2', \
                name_cat_metric: str = 'accuracy'):
    """
    Prova diferents imputadors, imprimeix els seus resultats i retorna el millor.
    En variables categòriques binaries, les següents mètriques es converteixen en mètriques binàries (ja que són més adequades per a aquest tipus de variables):
        - f1-micro --> f1-binary
        - f1-macro --> f1-binary
        - f1-weighted --> f1-binary
        - precision-micro --> precision-binary
        - precision-macro --> precision-binary
        - precision-weighted --> precision-binary
        - recall-micro --> recall-binary
        - recall-macro --> recall-binary
        - recall-weighted --> recall-binary
    """
    from sklearn.model_selection import KFold
    from sklearn.metrics import r2_score, accuracy_score, f1_score, precision_score, recall_score, mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
    from sklearn.impute import KNNImputer, SimpleImputer
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.linear_model import LinearRegression
    from sklearn.experimental import enable_iterative_imputer
    from sklearn.impute import IterativeImputer

    assert folds_cross_val > 1, "El paràmetre 'n_splits' ha de ser major que 1."

    categorical_binary_columns = ['Ascites', 'Hepatomegaly', 'Spiders']
    
    # Definir mètriques
    f1_micro = lambda y_true, y_pred: f1_score(y_true, y_pred, average='micro')
    f1_macro = lambda y_true, y_pred: f1_score(y_true, y_pred, average='macro')
    f1_weighted = lambda y_true, y_pred: f1_score(y_true, y_pred, average='weighted')
    precision_micro = lambda y_true, y_pred: precision_score(y_true, y_pred, average='micro')
    precision_macro = lambda y_true, y_pred: precision_score(y_true, y_pred, average='macro')
    precision_weighted = lambda y_true, y_pred: precision_score(y_true, y_pred, average='weighted')
    recall_micro = lambda y_true, y_pred: recall_score(y_true, y_pred, average='micro')
    recall_macro = lambda y_true, y_pred: recall_score(y_true, y_pred, average='macro')
    recall_weighted = lambda y_true, y_pred: recall_score(y_true, y_pred, average='weighted')

    fi_binary = lambda y_true, y_pred: f1_score(y_true, y_pred, average='binary')
    precision_binary = lambda y_true, y_pred: precision_score(y_true, y_pred, average='binary')
    recall_binary = lambda y_true, y_pred: recall_score(y_true, y_pred, average='binary')

    # Mapeig de les mètriques
    metric_functions = {
        'numerical': {
            'r2': r2_score, \
            'mse': mean_squared_error, \
            'mae': mean_absolute_error, \
            'mape': mean_absolute_percentage_error
        },
        'categorical': {
            'accuracy': accuracy_score, \
            'f1-micro': f1_micro, \
            'f1-macro': f1_macro, \
            'f1-weighted': f1_weighted, \
            'precision-micro': precision_micro, \
            'precision-macro': precision_macro, \
            'precision-weighted': precision_weighted, \
            'recall-micro': recall_micro, \
            'recall-macro': recall_macro, \
            'recall-weighted': recall_weighted
        }                
    }

    binary_categorical_functions_mapping= {
        'f1-micro': fi_binary, \
        'f1-macro': fi_binary, \
        'f1-weighted': fi_binary, \
        'precision-micro': precision_binary, \
        'precision-macro': precision_binary, \
        'precision-weighted': precision_binary, \
        'recall-micro': recall_binary, \
        'recall-macro': recall_binary, \
        'recall-weighted': recall_binary
    }

    assert name_num_metric in metric_functions['numerical'].keys(), f"El paràmetre 'name_num_metric' ha de ser una de les següents mètriques: {metric_functions['numerical'].keys()}"
    assert name_cat_metric in metric_functions['categorical'].keys(), f"El paràmetre 'name_cat_metric' ha de ser una de les següents mètriques: {metric_functions['categorical'].keys()}"

    num_metric = metric_functions['numerical'][name_num_metric]
    cat_metric = metric_functions['categorical'][name_cat_metric]

    # Definir les particions per a la validació creuada
    kf = KFold(n_splits=folds_cross_val, random_state=random_state, shuffle=True)
    
    # Inicialización de imputadores
    imputers = {
        'numerical': 
            {
            'KNNImputer-1': KNNImputer(n_neighbors=1),
            'KNNImputer-2': KNNImputer(n_neighbors=2),
            'KNNImputer-3': KNNImputer(n_neighbors=3),
            'KNNImputer-5': KNNImputer(n_neighbors=5),
            'KNNImputer-10': KNNImputer(n_neighbors=10),
            'KNNImputer-15': KNNImputer(n_neighbors=11),
            'KNNImputer-20': KNNImputer(n_neighbors=12),
            'KNNImputer-25': KNNImputer(n_neighbors=13),
            'KNNImputer-50': KNNImputer(n_neighbors=14),
            'SimpleImputer-mean': SimpleImputer(strategy='mean'),
            'IterativeImputer-lr': IterativeImputer(estimator=LinearRegression(fit_intercept=True), random_state=random_state),
        },

        'categorical': 
            {
            'KNeighborsClassifier-1': KNeighborsClassifier(n_neighbors=1),
            'KNeighborsClassifier-2': KNeighborsClassifier(n_neighbors=2),
            'KNeighborsClassifier-3': KNeighborsClassifier(n_neighbors=3),
            'KNeighborsClassifier-5': KNeighborsClassifier(n_neighbors=5),
            'KNeighborsClassifier-10': KNeighborsClassifier(n_neighbors=10),
            'KNeighborsClassifier-15': KNeighborsClassifier(n_neighbors=15),
            'KNeighborsClassifier-20': KNeighborsClassifier(n_neighbors=20),
            'KNeighborsClassifier-25': KNeighborsClassifier(n_neighbors=25),
            'KNeighborsClassifier-50': KNeighborsClassifier(n_neighbors=50),
            'DecisionTreeClassifier-gini': DecisionTreeClassifier(criterion='gini', random_state=random_state),
            'DecisionTreeClassifier-entropy': DecisionTreeClassifier(criterion='entropy', random_state=random_state), 
            'RandomForestClassifier-gini': RandomForestClassifier(criterion='gini', random_state=random_state),
            'RandomForestClassifier-entropy': RandomForestClassifier(criterion='entropy', random_state=random_state),
        }
    }

    # Inicialització dels resultats
    scores = {'numerical': {name_num_imputer: [] for name_num_imputer in imputers['numerical'].keys()},
              'categorical': {name_cat_imputer: [] for name_cat_imputer in imputers['categorical'].keys()}}
    
    # Treiem les files amb NaNs
    X_train_complete = X_train.dropna()

    numerical_columns = X_train_complete.select_dtypes(include=['Int64', 'float64']).columns
    categorical_columns = X_train_complete.select_dtypes(include=['category']).columns

    for train_index, test_index in kf.split(X_train_complete):
        X_train_fold, X_test_fold = X_train_complete.iloc[train_index], X_train_complete.iloc[test_index]

        # Imputació i evaluació per a numèriques
        for name, imputer in imputers['numerical'].items():
            # Introdir artificalment NaNs
            X_test_fold_num_with_nan = X_test_fold.copy()
            for index in X_test_fold.index:
                # Seleccionar aleatoriament una columna numèrica i una categòrica
                np.random.seed(random_state)
                num_col = np.random.choice(numerical_columns)
                X_test_fold_num_with_nan.loc[index, num_col] = np.nan # Només NaNs en les columnes numèriques

            imputer.fit(X_train_fold[numerical_columns])
            X_test_num_imputed = imputer.transform(X_test_fold_num_with_nan[numerical_columns])
            X_test_num_imputed = pd.DataFrame(X_test_num_imputed, columns=numerical_columns, index=X_test_fold_num_with_nan.index)

            num_cols_scores = []
            for col in numerical_columns:
                if X_test_fold_num_with_nan[col].isna().any():
                    # Només evaluar les columnes amb NaNs
                    true_values = X_test_fold[col]
                    imputed_values = X_test_num_imputed[col]
                    score = num_metric(true_values, imputed_values)
                    num_cols_scores.append(score)
                
            scores['numerical'][name].append(np.mean(num_cols_scores))
                
        # Imputació i evaluació per a categòriques
        for name, imputer in imputers['categorical'].items():
            # Introdir artificalment NaNs
            X_test_fold_cat_with_nan = X_test_fold.copy()
            for index in X_test_fold.index:
                # Seleccionar aleatoriament una columna numèrica i una categòrica
                np.random.seed(random_state)
                cat_col = np.random.choice(categorical_columns)
                X_test_fold_cat_with_nan.loc[index, cat_col] = np.nan # Només NaNs en les columnes categòriques

            cat_cols_scores = []
            for col in categorical_columns:
                if col in categorical_binary_columns and name_cat_metric in binary_categorical_functions_mapping.keys():
                    cat_metric = binary_categorical_functions_mapping[name_cat_metric]
                else:
                    cat_metric = metric_functions['categorical'][name_cat_metric]
                imputer.fit(X_train_fold[numerical_columns], X_train_fold[col])
                X_test_cat_col_imputed = imputer.predict(X_test_fold_cat_with_nan[numerical_columns])
                X_test_cat_col_imputed = pd.Series(X_test_cat_col_imputed, index=X_test_fold_cat_with_nan.index)

                true_values = X_test_fold[col]
                predicted_values = X_test_cat_col_imputed
                score = cat_metric(true_values, predicted_values)
                cat_cols_scores.append(score)

            scores['categorical'][name].append(np.mean(cat_cols_scores))

    # Calcular la mitjana dels resultats
    average_scores = {
        'numerical': {name: np.mean(scores_val) for name, scores_val in scores['numerical'].items()},
        'categorical': {name: np.mean(scores_val) for name, scores_val in scores['categorical'].items()}
    }
    
    # Guardar el millor imputador numèric
    best_num_imputer_name = 'None'
    best_num_imputer_score = float('-inf') if name_num_metric not in {'mse', 'mae', 'mape'} else float('inf')
    num_condition = lambda x, y: x > y if name_num_metric not in {'mse', 'mae', 'mape'} else x < y # Si la mètrica és mse, mae o mape el millor imputador és el que tingui el valor més petit
    for name_num_imputer, score_num_imputer in average_scores['numerical'].items():
        if num_condition(score_num_imputer, best_num_imputer_score):
            best_num_imputer_score = score_num_imputer
            best_num_imputer_name = name_num_imputer

    # Guardar el millor imputador categòric
    best_cat_imputer_name = 'None'
    best_cat_imputer_score = float('-inf')
    for name_cat_imputer, score_cat_imputer in average_scores['categorical'].items():
        if score_cat_imputer > best_cat_imputer_score:
            best_cat_imputer_score = score_cat_imputer
            best_cat_imputer_name = name_cat_imputer

    # Mostrar resultats
    if print_scores:
        for name_num_imputer, score_num_imputer in average_scores['numerical'].items():
            print(f"NUMERICAL IMPUTER [{name_num_imputer}] --> {score_num_imputer} (\'{name_num_metric}\' score)")

        for name_cat_imputer, score_cat_imputer in average_scores['categorical'].items():
            print(f"CATEGORICAL IMPUTER [{name_cat_imputer}] --> {score_cat_imputer} (\'{name_cat_metric}\' score)")
        
        print(f"\nBEST NUMERICAL IMPUTER: {best_num_imputer_name} --> {best_num_imputer_score} (\'{name_num_metric}\' score)")
        print(f"BEST CATEGORICAL IMPUTER: {best_cat_imputer_name} --> {best_cat_imputer_score} (\'{name_cat_metric}\' score)")

        if name_num_metric not in {'mse', 'mae', 'mape'}:
            print(f"BEST COMBINED IMPUTER: {best_num_imputer_name} (num) + {best_cat_imputer_name} (cat) --> {np.mean([best_num_imputer_score, best_cat_imputer_score])} (combined \'{name_num_metric}\' & \'{name_cat_metric}\' scores)")
    
    # Retornar els millors imputadors i les seves puntuacions
    return {'numerical': {'name': best_num_imputer_name, 'imputer': imputers['numerical'][best_num_imputer_name], 'score': best_num_imputer_score},
        'categorical': {'name': best_cat_imputer_name, 'imputer': imputers['categorical'][best_cat_imputer_name], 'score': best_cat_imputer_score},
        'combined': {'name': f"{best_num_imputer_name} (num) + {best_cat_imputer_name} (cat)", 'score': np.mean([best_num_imputer_score, best_cat_imputer_score])}}

#best_imputers = find_best_imputer(X_train=train, n_splits=5, random_state=42, print_scores=True, name_num_metric='r2', name_cat_metric='accuracy')

In [19]:
def impute_data(data_to_impute: pd.DataFrame, \
				numerical_imputer = 'best', \
				categorical_imputer = 'best', \
				save_to_csv: bool = True, \
				random_state: int = 42, \
				folds_cross_val: int = 5, \
				num_metric = 'r2', \
				cat_metric = 'accuracy'):
	"""
	Imputa els valors NaN del dataset.
	"""
	from sklearn.impute import KNNImputer, SimpleImputer
	from sklearn.neighbors import KNeighborsClassifier
	from sklearn.tree import DecisionTreeClassifier
	from sklearn.ensemble import RandomForestClassifier
	from sklearn.linear_model import LinearRegression
	from sklearn.experimental import enable_iterative_imputer
	from sklearn.impute import IterativeImputer
	
	imputers = {
        'numerical': 
            {
            'KNNImputer-1': KNNImputer(n_neighbors=1),
            'KNNImputer-2': KNNImputer(n_neighbors=2),
            'KNNImputer-3': KNNImputer(n_neighbors=3),
            'KNNImputer-5': KNNImputer(n_neighbors=5),
            'KNNImputer-10': KNNImputer(n_neighbors=10),
            'KNNImputer-15': KNNImputer(n_neighbors=11),
            'KNNImputer-20': KNNImputer(n_neighbors=12),
            'KNNImputer-25': KNNImputer(n_neighbors=13),
            'KNNImputer-50': KNNImputer(n_neighbors=14),
            'SimpleImputer-mean': SimpleImputer(strategy='mean'),
            'IterativeImputer-lr': IterativeImputer(estimator=LinearRegression(fit_intercept=True), random_state=random_state),
        },

        'categorical': 
            {
            'KNeighborsClassifier-1': KNeighborsClassifier(n_neighbors=1),
            'KNeighborsClassifier-2': KNeighborsClassifier(n_neighbors=2),
            'KNeighborsClassifier-3': KNeighborsClassifier(n_neighbors=3),
            'KNeighborsClassifier-5': KNeighborsClassifier(n_neighbors=5),
            'KNeighborsClassifier-10': KNeighborsClassifier(n_neighbors=10),
            'KNeighborsClassifier-15': KNeighborsClassifier(n_neighbors=15),
            'KNeighborsClassifier-20': KNeighborsClassifier(n_neighbors=20),
            'KNeighborsClassifier-25': KNeighborsClassifier(n_neighbors=25),
            'KNeighborsClassifier-50': KNeighborsClassifier(n_neighbors=50),
            'DecisionTreeClassifier-gini': DecisionTreeClassifier(criterion='gini', random_state=random_state),
            'DecisionTreeClassifier-entropy': DecisionTreeClassifier(criterion='entropy', random_state=random_state), 
            'RandomForestClassifier-gini': RandomForestClassifier(criterion='gini', random_state=random_state),
            'RandomForestClassifier-entropy': RandomForestClassifier(criterion='entropy', random_state=random_state),
        }
    }

	possible_numerical_imputers = imputers['numerical'].keys()
	assert numerical_imputer in possible_numerical_imputers or numerical_imputer == 'best', f"numerical_imputer ha de ser un dels següents valors: {possible_numerical_imputers} (o 'best' per trobar el millor)"
	possible_categorical_imputers = imputers['categorical'].keys()
	assert categorical_imputer in possible_categorical_imputers or categorical_imputer == 'best', f"categorical_imputer ha de ser un dels següents valors: {possible_categorical_imputers} (o 'best' per trobar el millor)"

	if numerical_imputer == 'best' or categorical_imputer == 'best':
		best_imputers_dict = find_best_imputer(X_train=X_train, \
										random_state=random_state, \
										print_scores=False, \
										name_num_metric=num_metric, \
										name_cat_metric=cat_metric, \
										folds_cross_val=folds_cross_val)

		if numerical_imputer == 'best' and categorical_imputer == 'best':
			numerical_imputer = best_imputers_dict['numerical']['imputer']
			categorical_imputer = best_imputers_dict['categorical']['imputer']
			print(f"IMPUTER COMBINAT ESCOLLIT: {best_imputers_dict['combined']['name']} --> {best_imputers_dict['combined']['score']} (combinació de \'{num_metric}\' i \'{cat_metric}\' en X_train)")
		else:
			if numerical_imputer == 'best':
				numerical_imputer = best_imputers_dict['numerical']['imputer']
				print(f"IMPUTER NUMÈRIC ESCOLLIT: {best_imputers_dict['numerical']['name']} --> {best_imputers_dict['numerical']['score']} (\'{num_metric}\' en X_train)")
			else:
				numerical_imputer = imputers['numerical'][numerical_imputer]

			if categorical_imputer == 'best':
				categorical_imputer = best_imputers_dict['categorical']['imputer']
				print(f"IMPUTER CATEGÒRIC ESCOLLIT: {best_imputers_dict['categorical']['name']} --> {best_imputers_dict['categorical']['score']} (\'{cat_metric}\' en X_train)")
			else:
				categorical_imputer = imputers['categorical'][categorical_imputer]
	
	else:
		numerical_imputer = imputers['numerical'][numerical_imputer]
		categorical_imputer = imputers['categorical'][categorical_imputer]


	X_train_complete = X_train.dropna()

	numerical_cols = data_to_impute.select_dtypes(include=['Int64', 'float64']).columns
	categorical_cols = data_to_impute.select_dtypes(include=['category']).columns


	# Imputar numèriques
	numerical_imputer.fit(X_train_complete[numerical_cols])
	imputed_num_cols = numerical_imputer.transform(data_to_impute[numerical_cols])
	imputed_num_cols = pd.DataFrame(imputed_num_cols, columns=numerical_cols, index=data_to_impute.index)
	data_to_impute[numerical_cols] = imputed_num_cols

	# Imputar categòriques
	for col in categorical_cols:
		if data_to_impute[col].isna().any():
			categorical_imputer.fit(X_train_complete[numerical_cols], X_train_complete[col])
			imputed_cat_col = categorical_imputer.predict(data_to_impute[numerical_cols])
			imputed_cat_col = pd.Series(imputed_cat_col, index=data_to_impute[col].index)
			data_to_impute[col] = imputed_cat_col

	# Comprovar que no queden NaNs
	assert not data_to_impute.isna().any().any(), "No s'han pogut eliminar tots els NaNs del dataset."

	# Guardar el dataset
	if save_to_csv:
		data_to_impute.to_csv('../assets/data/imputed_cirrhosis.csv', index=False)

#impute_data(data_to_impute=X_train, imputer='best', save_to_csv=True, random_state=42, encode=True, decode=True)

### **Balanceig de classes de la variable objectiu (Status)**

In [36]:
def balance_target_classes(X_data: pd.DataFrame, y_data: pd.Series, method: str ='oversample', random_state: int = 42):
	"""
	Balanceja les classes de la variable target utilitzant el mètode especificat. X_data no han de contenir NaNs.
	"""
	from imblearn.over_sampling import RandomOverSampler, SMOTE
	from imblearn.under_sampling import RandomUnderSampler
	from imblearn.combine import SMOTEENN
	
	methods = {
		'oversample': RandomOverSampler(random_state=random_state),
		'undersample': RandomUnderSampler(random_state=random_state),
		'smote': SMOTE(random_state=random_state),
		'smoteenn': SMOTEENN(random_state=random_state)
	}

	assert method in methods.keys(), f"El paràmetre 'method' ha de ser un dels següents: {methods.keys()}"

	encode = True if method in {'smote', 'smoteenn'} else False

	if encode:
		# Cal fer OneHotEncoding a les variables categòriques perquè SMOTE o SMOTEENN les puguin tractar
		encode_variables(data=X_data, save_to_csv=False)

	print(f"Mostres per classe abans de balancejar ({method}):")
	for class_, count in y_data.value_counts().items():
		print(f"\t*{class_}: {count}")
	print(f"\t*Total de mostres: {len(y_data)}")

	# Seleccionar el mètode de balanceig
	balancer = methods[method]

	# Balancejar les classes
	X_balanced, y_balanced = balancer.fit_resample(X_data, y_data)

	# Convertir a DataFrame i Series
	X_balanced = pd.DataFrame(X_balanced, columns=X_data.columns)
	y_balanced = pd.Series(y_balanced, name=y_data.name)

	if encode:
		# Decodificar les variables categòriques
		decode_variables(data=X_balanced, ohe_mapping=ohe_mapping, original_columns_order=original_columns_order)
		
	# Guardar el dataset en l'entorn global
	global X_train, y_train
	X_train = X_balanced
	y_train = y_balanced
	
	print(f"Mostres per classe després de balancejar ({method}):")
	for class_, count in y_balanced.value_counts().items():
		print(f"\t*{class_}: {count}")
	print(f"\t*Total de mostres: {len(y_balanced)}")

# balance_target_classes(X_data=X_train, y_data=y_train, method='smoteenn', random_state=random_state)

### **Correlacions entre variables numèriques**

In [ ]:
def numerical_vars_correlations(data: pd.DataFrame):
	"""
	Visualitza la correlació entre les variables numèriques.
	"""

	numerical_columns = data.select_dtypes(include=['Int64', 'float64']).columns

	plt.figure(figsize=(10, 6))

	sns.heatmap(data[numerical_columns].corr(), annot=True, cmap='coolwarm', vmin=-1, vmax=1)

	plt.title('Correlació entre les variables numèriques')
	plt.tight_layout()
	plt.show()


numerical_vars_correlations(data=data)

### **Correlacions entre variables categòriques i variable objectiu**

In [ ]:
def categorical_target_relationships(data: pd.DataFrame):
    """
    Visualitza la relació entre les variables categòriques i la variable target.
    """
    categorical_columns = data.select_dtypes(include=['category']).columns

    for col in categorical_columns:
        plt.figure(figsize=(10, 4))
        sns.countplot(data=data, x=col, hue='Status')
        plt.title(f'Distribució de {col} vs Status (target)')
        plt.show()

categorical_target_relationships(data=train)

### **Anàlisis de Components Principals (ACP)**

In [ ]:
def principal_component_analysis(data: pd.DataFrame):
	"""
	Aplica PCA para explicar el 80% de la varianza y visualiza los resultados.

	:param X: DataFrame con los datos numéricos.
	:param categorical_columns: Lista de nombres de las columnas categóricas para análisis de centroides.
	"""
	from sklearn.decomposition import PCA
	# Seleccionar variables numéricas
	numerical_columns = data.select_dtypes(include=['Int64', 'float64']).columns
	categorical_columns = data.select_dtypes(include=['category']).columns

	# Aplicar PCA
	pca = PCA(n_components=0.80)
	X_pca = pca.fit_transform(data[numerical_columns])

	# Variància acumulada
	plt.figure(figsize=(10, 6))
	plt.plot(np.cumsum(pca.explained_variance_ratio_))
	plt.xlabel('Número de components')
	plt.ylabel('Variància acumulada')
	plt.title('Anàlisi de de la variància acumulada en funció del número de components')
	plt.grid(True)
	plt.show()

	# Gràfic de barres amb una línia que marca el 80% de la variància
	plt.figure(figsize=(10, 6))
	plt.bar(range(len(pca.explained_variance_ratio_)), pca.explained_variance_ratio_)
	plt.plot(range(len(pca.explained_variance_ratio_)), np.cumsum(pca.explained_variance_ratio_), c='red')
	plt.xlabel('Número de components')
	plt.ylabel('Variància')
	plt.title('Anàlisi de la variància en funció del número de components')
	plt.grid(True)
	plt.show()

	# Projecció de les variables numèriques
	plt.figure(figsize=(10, 6))
	plt.scatter(X_pca[:, 0], X_pca[:, 1])
	inercia_primer_comp = round(pca.explained_variance_ratio_[0] * 100, 2)
	inercia_segon_comp= round(pca.explained_variance_ratio_[1] * 100, 2)
	plt.xlabel(f'1r component principal ({inercia_primer_comp}%)')
	plt.ylabel(f'2n component principal ({inercia_segon_comp}%)')
	plt.title('Projecció de les variables numèriques sobre els dos primers components principals')
	plt.grid(True)

	# Projecció de les variables categòriques
	for col in categorical_columns:
		if col in data.columns:
			for category in np.unique(data[col]):
				mask = data[col] == category
				centroid = np.mean(X_pca[mask], axis=0)
				plt.scatter(centroid[0], centroid[1], marker='x', color='red')
	plt.legend()
	plt.show()

principal_component_analysis(data=train.dropna())

### **1r Model: K-Nearest Neighbors (KNN)**

In [13]:
def knn_prediction(X_train: pd.DataFrame, y_train: pd.Series, X_test: pd.DataFrame, y_test: pd.Series, \
					k_list: list[int] = [1, 2, 3, 5, 10, 15, 20, 25, 50], \
					scorer_cross_val: str = 'accuracy', \
					folds_cross_val: int = 5, \
					plot_confusion_matrix: bool = True, \
					print_scores: bool = True, \
					encode: bool = False):
	"""
	Busca el millor valor de k per a KNN utilitzant cross validation i mostra els resultats.
	"""
	from sklearn.neighbors import KNeighborsClassifier
	from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, ConfusionMatrixDisplay, make_scorer
	from sklearn.model_selection import cross_val_score

	assert all([k > 0 for k in k_list]), "Els valors de k han de ser positius."
	assert folds_cross_val > 1, "El paràmetre 'folds_cross_val' ha de ser major que 1."

	scorers_cross_val = {
		'accuracy': make_scorer(accuracy_score),
		'f1-micro': make_scorer(f1_score, average='micro'),
		'f1-macro': make_scorer(f1_score, average='macro'),
		'f1-weighted': make_scorer(f1_score, average='weighted'),
		'precision-micro': make_scorer(precision_score, average='micro'),
		'precision-macro': make_scorer(precision_score, average='macro'),
		'precision-weighted': make_scorer(precision_score, average='weighted'),
		'recall-micro': make_scorer(recall_score, average='micro'),
		'recall-macro': make_scorer(recall_score, average='macro'),
		'recall-weighted': make_scorer(recall_score, average='weighted')
	}

	assert scorer_cross_val in scorers_cross_val.keys(), f"El paràmetre 'scorer_cross_val' ha de ser un dels següents: {scorers_cross_val.keys()}"

	# Encode
	if encode:
		encode_variables(data=X_train, save_to_csv=True)
		encode_variables(data=X_test, save_to_csv=False)
		X_train_ready = X_train
		X_test_ready = X_test
	else:
		# Només variables numèriques per a KNN
		numerical_columns = X_train.select_dtypes(include=['Int64', 'float64']).columns
		X_train_ready = X_train[numerical_columns]
		X_test_ready = X_test[numerical_columns]
			
	# Cross validation per a cada valor de k
	scores_cross_val = {}
	for k in k_list:
		knn = KNeighborsClassifier(n_neighbors=k)
		scores_cross_val[k] = np.mean([cross_val_score(knn, X_train_ready, y_train, cv=folds_cross_val, scoring=scorers_cross_val[scorer_cross_val])])

	best_k = max(scores_cross_val, key=lambda k: scores_cross_val[k])
	best_knn = KNeighborsClassifier(n_neighbors=best_k)

	if print_scores:
		print("\nMODEL K-NEAREST NEIGHBORS (KNN)")
		print(f"Millors Paràmetres Cross Validation: k={best_k}")
		print(f"Millor Score Cross Validation ({scorer_cross_val}) --> {scores_cross_val[best_k]}")

	# Realitzem la predicció
	best_knn.fit(X_train_ready, y_train)
	y_pred = best_knn.predict(X_test_ready)

	# Resultats
	scores = {
		'accuracy': accuracy_score(y_test, y_pred),
		'f1-micro': f1_score(y_test, y_pred, average='micro'),
		'f1-macro': f1_score(y_test, y_pred, average='macro'),
		'f1-weighted': f1_score(y_test, y_pred, average='weighted'),
		'precision-micro': precision_score(y_test, y_pred, average='micro'),
		'precision-macro': precision_score(y_test, y_pred, average='macro'),
		'precision-weighted': precision_score(y_test, y_pred, average='weighted'),
		'recall-micro': recall_score(y_test, y_pred, average='micro'),
		'recall-macro': recall_score(y_test, y_pred, average='macro'),
		'recall-weighted': recall_score(y_test, y_pred, average='weighted')
	}

	# Mostrar els resultats
	if print_scores:
		print(f"SCORES BEST KNN (k={best_k})")
		for metric in scores:
			print(f"\t*{metric} --> {scores[metric]}")
		print(f"\t*Average Metrics Score --> {np.mean(list(scores.values()))}")
	
	# Decode
	if encode:
		decode_variables(data=X_train, ohe_mapping=ohe_mapping, original_columns_order=original_columns_order)
		decode_variables(data=X_test, ohe_mapping=ohe_mapping, original_columns_order=original_columns_order)

	if plot_confusion_matrix:
		# Matriu de confusió 
		unique_classes = np.unique(y_test)
		cm = confusion_matrix(y_test, y_pred, labels=unique_classes)
		ConfusionMatrixDisplay(cm, display_labels=unique_classes).plot()
		plt.title(f"Matriu de Confusió - KNN (k={best_k})")
		plt.show()

	return scores, best_k

### **2n Model: Decision Tree**

In [14]:
def dt_prediction(X_train: pd.DataFrame, y_train: pd.Series, X_test: pd.DataFrame, y_test: pd.Series,
				  	criterion_list: list[str] = ['entropy', 'gini'], \
					max_depth_list: list[int|None] = [None, 2, 3, 5, 10, 15, 20, 25, 50],
				  	min_samples_split_list: list[int] = [2, 3, 5, 10, 15, 20, 25, 50], \
					min_samples_leaf_list: list[int] = [1, 2, 3, 5, 10, 15, 20, 25, 50],
				  	random_state: int = 42, \
					folds_cross_val: int = 5, \
					scorer_cross_val: str = 'accuracy',
				  	plot_confusion_matrix: bool = True, \
					print_scores: bool = True):
	"""
	Busca el millor model de Decision Tree utilitzant la validació creuada i fa la predicció amb els millors paràmetres.
	"""
	from sklearn.tree import DecisionTreeClassifier
	from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, ConfusionMatrixDisplay, make_scorer
	from sklearn.model_selection import GridSearchCV

	assert all([max_depth is None or max_depth > 0 for max_depth in max_depth_list]), "Els valors de max_depth han de ser positius o None."
	assert all([min_samples_split > 1 for min_samples_split in min_samples_split_list]), "Els valors de min_samples_split han de ser majors que 1."
	assert all([min_samples_leaf > 0 for min_samples_leaf in min_samples_leaf_list]), "Els valors de min_samples_leaf han de ser positius."
	assert folds_cross_val > 1, "El paràmetre 'cross_val_folds' ha de ser major que 1."
	assert all([criterion in {'entropy', 'gini'} for criterion in criterion_list]), "Els valors de criterion han de ser 'entropy' o 'gini'."

	scorers_cross_val = {
		'accuracy': make_scorer(accuracy_score),
		'f1-micro': make_scorer(f1_score, average='micro'),
		'f1-macro': make_scorer(f1_score, average='macro'),
		'f1-weighted': make_scorer(f1_score, average='weighted'),
		'precision-micro': make_scorer(precision_score, average='micro'),
		'precision-macro': make_scorer(precision_score, average='macro'),
		'precision-weighted': make_scorer(precision_score, average='weighted'),
		'recall-micro': make_scorer(recall_score, average='micro'),
		'recall-macro': make_scorer(recall_score, average='macro'),
		'recall-weighted': make_scorer(recall_score, average='weighted')
	}

	assert scorer_cross_val in scorers_cross_val.keys(), f"El paràmetre 'scorer_cross_val' ha de ser un dels següents: {scorers_cross_val.keys()}"

	# Configurar la cerca dels millors paràmetres
	param_grid = {
		'criterion': criterion_list,
		'max_depth': max_depth_list or [None],
		'min_samples_split': min_samples_split_list or [2],
		'min_samples_leaf': min_samples_leaf_list or [1]
	}

	dt = DecisionTreeClassifier(random_state=random_state)	
	grid_search = GridSearchCV(dt, param_grid, cv=folds_cross_val, scoring=scorers_cross_val[scorer_cross_val])
	grid_search.fit(X_train, y_train)

	best_dt = grid_search.best_estimator_
	best_params = grid_search.best_params_

	if print_scores:
		print("\nMODEL DECISION TREE")
		print(f"Millors Paràmetres Cross Validation: {best_params}")
		print(f"Millor Score Cross Validation ({scorer_cross_val}) --> {grid_search.best_score_}")

	# Predicció del millor model
	y_pred = best_dt.predict(X_test)

	# Resultats
	scores = {
		'accuracy': accuracy_score(y_test, y_pred),
		'f1-micro': f1_score(y_test, y_pred, average='micro'),
		'f1-macro': f1_score(y_test, y_pred, average='macro'),
		'f1-weighted': f1_score(y_test, y_pred, average='weighted'),
		'precision-micro': precision_score(y_test, y_pred, average='micro'),
		'precision-macro': precision_score(y_test, y_pred, average='macro'),
		'precision-weighted': precision_score(y_test, y_pred, average='weighted'),
		'recall-micro': recall_score(y_test, y_pred, average='micro'),
		'recall-macro': recall_score(y_test, y_pred, average='macro'),
		'recall-weighted': recall_score(y_test, y_pred, average='weighted')
	}

	# Mostrar resultats
	if print_scores:
		print(f"SCORES BEST DECISION TREE ({best_params})")
		for metric, score in scores.items():
			print(f"\t*{metric}: {score}")

	# Mostrar la matriu de confusió
	if plot_confusion_matrix:
		unique_classes = np.unique(y_test)
		cm = confusion_matrix(y_test, y_pred, labels=unique_classes)
		ConfusionMatrixDisplay(cm, display_labels=unique_classes).plot()
		plt.title(f"Matriu de Confusió - Decision Tree ({best_params})")
		plt.show()

	return scores, best_params

### **3r Model: Support Vector Machine (SVM)**

In [15]:
def svm_prediction(X_train: pd.DataFrame, y_train: pd.Series, X_test: pd.DataFrame, y_test: pd.Series, \
					kernel_list: list[str] = ['linear', 'poly', 'rbf', 'sigmoid'], \
					C_list: list[float] = [0.1, 0.5, 1, 2, 5, 10, 20, 50, 100], \
					gamma_list: list[str] = ['scale', 'auto'], \
					scorer_cross_val: str = 'accuracy', \
					folds_cross_val: int = 5, \
					print_scores: bool = True, \
					plot_confusion_matrix: bool = True, \
					random_state: int = 42):
	"""
	Busca el millor model SVM utilitzant la validació creuada i fa la predicció amb els millors paràmetres.
	"""
	from sklearn.svm import SVC
	from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, ConfusionMatrixDisplay, make_scorer
	from sklearn.model_selection import GridSearchCV

	assert all([C > 0 for C in C_list]), "Els valors de C han de ser positius."
	assert folds_cross_val > 1, "El paràmetre 'cross_val_folds' ha de ser major que 1."
	assert all([kernel in {'linear', 'poly', 'rbf', 'sigmoid'} for kernel in kernel_list]), "Els valors de kernel han de ser 'linear', 'poly', 'rbf' o 'sigmoid'."
	assert all([gamma in {'scale', 'auto'} for gamma in gamma_list]), "Els valors de gamma han de ser 'scale' o 'auto'."

	scorers_cross_val = {
		'accuracy': make_scorer(accuracy_score),
		'f1-micro': make_scorer(f1_score, average='micro'),
		'f1-macro': make_scorer(f1_score, average='macro'),
		'f1-weighted': make_scorer(f1_score, average='weighted'),
		'precision-micro': make_scorer(precision_score, average='micro'),
		'precision-macro': make_scorer(precision_score, average='macro'),
		'precision-weighted': make_scorer(precision_score, average='weighted'),
		'recall-micro': make_scorer(recall_score, average='micro'),
		'recall-macro': make_scorer(recall_score, average='macro'),
		'recall-weighted': make_scorer(recall_score, average='weighted')
	}

	assert scorer_cross_val in scorers_cross_val.keys(), f"El paràmetre 'scorer_cross_val' ha de ser un dels següents: {scorers_cross_val.keys()}"

	# Configurar la cerca dels millors paràmetres
	param_grid = {
		'kernel': kernel_list,
		'C': C_list,
		'gamma': gamma_list
	}

	svm = SVC(random_state=random_state)
	grid_search = GridSearchCV(svm, param_grid, cv=folds_cross_val, scoring=scorers_cross_val[scorer_cross_val])
	grid_search.fit(X_train, y_train)

	best_svm = grid_search.best_estimator_
	best_params = grid_search.best_params_

	if print_scores:
		print("\nMODEL SUPPORT VECTOR MACHINE (SVM)")
		print(f"Millors Paràmetres Cross Validation: {best_params}")
		print(f"Millor Score Cross Validation ({scorer_cross_val}) --> {grid_search.best_score_}")

	# Predicció del millor model
	y_pred = best_svm.predict(X_test)

	# Resultats
	scores = {
		'accuracy': accuracy_score(y_test, y_pred),
		'f1-micro': f1_score(y_test, y_pred, average='micro'),
		'f1-macro': f1_score(y_test, y_pred, average='macro'),
		'f1-weighted': f1_score(y_test, y_pred, average='weighted'),
		'precision-micro': precision_score(y_test, y_pred, average='micro'),
		'precision-macro': precision_score(y_test, y_pred, average='macro'),
		'precision-weighted': precision_score(y_test, y_pred, average='weighted'),
		'recall-micro': recall_score(y_test, y_pred, average='micro'),
		'recall-macro': recall_score(y_test, y_pred, average='macro'),
		'recall-weighted': recall_score(y_test, y_pred, average='weighted')
	}

	# Mostrar resultats
	if print_scores:
		print(f"SCORES BEST SVM ({best_params})")
		for metric, score in scores.items():
			print(f"\t*{metric}: {score}")

	# Mostrar la matriu de confusió
	if plot_confusion_matrix:
		unique_classes = np.unique(y_test)
		cm = confusion_matrix(y_test, y_pred, labels=unique_classes)
		ConfusionMatrixDisplay(cm, display_labels=unique_classes).plot()
		plt.title(f"Matriu de Confusió - SVM ({best_params})")
		plt.show()

	return scores, best_params	


In [38]:
random_state = 42
#random_state = np.random.randint(0, 1000)
print(f"Random state: {random_state}")
num_metric = 'r2'
cat_metric = 'f1-weighted'
folds = 5
load_dataset(save_to_csv=False)
initial_preprocessing(data=data, save_to_csv=False)
#delete_outliers(data=data, factor=3, plots=False, save_to_csv=False)
#delete_last_rows(data=data, save_to_csv=False)
split_dataset(data=data, test_size=0.15, random_state=random_state)
scale_variables(data=X_train, scaler='minmax', save_to_csv=False)
scale_variables(data=X_test, scaler='minmax', save_to_csv=False)
impute_data(data_to_impute=X_test, \
			numerical_imputer='best', \
			categorical_imputer='best', \
			save_to_csv=False, \
			random_state=random_state, \
			folds_cross_val=folds, \
			num_metric=num_metric, \
			cat_metric=cat_metric)
impute_data(data_to_impute=X_train, \
			numerical_imputer='best', \
			categorical_imputer='best', \
			save_to_csv=False, \
			random_state=random_state, \
			folds_cross_val=folds, \
			num_metric=num_metric, \
			cat_metric=cat_metric)
assert not X_train.isna().any().any(), "ABANS ENCODE 1."
encode_variables(data=X_train, save_to_csv=False)
assert not X_train.isna().any().any(), "DESPRES ENCODE 1."
decode_variables(data=X_train, ohe_mapping=ohe_mapping, original_columns_order=original_columns_order)
assert not X_train.isna().any().any(), "DESPRES DECODE 1."
balance_target_classes(X_data=X_train, y_data=y_train, method='smote', random_state=random_state)
assert not X_train.isna().any().any(), "DESPRES BALANCE."

_ = knn_prediction(X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test, \
			   	k_list=[1, 2, 3, 5, 10, 15, 20, 25, 50], \
				scorer_cross_val=cat_metric, \
				folds_cross_val=folds, \
				encode=False, \
				plot_confusion_matrix=True, \
				print_scores=True)
#_ = dt_prediction(X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test, \criterion_list=['entropy', 'gini'], max_depth_list=[None, 2, 3, 5, 10, 15, 20, 25, 50], min_samples_split_list=[2, 3, 5, 10, 15, 20, 25, 50], min_samples_leaf_list=[1, 2, 3, 5, 10, 15, 20, 25, 50], random_state=random_state, folds_cross_val=folds, scorer_cross_val=cat_metric, plot_confusion_matrix=False, print_scores=True)
#_ = svm_prediction(X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test, kernel_list=['linear', 'poly', 'rbf', 'sigmoid'], C_list=[0.1, 0.5, 1, 2, 5, 10, 20, 50, 100], gamma_list=['scale', 'auto'], scorer_cross_val='cat_metric', folds_cross_val=folds, print_scores=True, plot_confusion_matrix=False, random_state=random_state)


Random state: 42
Train shape: (355, 19)
Test shape: (63, 19)
IMPUTER COMBINAT ESCOLLIT: KNNImputer-50 (num) + RandomForestClassifier-gini (cat) --> 0.3244969179652108 (combinació de 'r2' i 'f1-weighted' en X_train)
IMPUTER COMBINAT ESCOLLIT: KNNImputer-50 (num) + RandomForestClassifier-gini (cat) --> 0.3244969179652108 (combinació de 'r2' i 'f1-weighted' en X_train)
Mostres per classe abans de balancejar (smote):
	*Alive: 197
	*Dead: 137
	*LiverTransplant: 21
	*Total de mostres: 355
Mostres per classe després de balancejar (smote):
	*Alive: 197
	*LiverTransplant: 197
	*Dead: 197
	*Total de mostres: 591


AssertionError: DESPRES BALANCE.

### **Execució d'experiments**

In [ ]:
def run_experiment(
		random_state: int = 42,
		remove_outliers: bool = True,
		outliers_factor: int = 3,
		remove_last_rows: bool = False,
		test_size: float = 0.15,
		scaler: str = '',
		imputer_cross_val_folds: int = 5,
		imputer_num_metric: str = 'r2',
		imputer_cat_metric: str = 'accuracy',
		balancer: str = '',
		predictor: str = 'knn',
		knn_params: dict = {'k_list': [1, 2, 3, 5, 10, 15, 20, 25, 50], \
					  		'scorer_cross_val': 'accuracy', \
							'folds_cross_val': 5, \
							'encode': False, \
							'plot_confusion_matrix': True, \
							'print_scores': True},
		dt_params: dict = {'criterion_list': ['entropy', 'gini'], \
							'max_depth_list': [None, 2, 3, 5, 10, 15, 20, 25, 50], \
							'min_samples_split_list': [2, 3, 5, 10, 15, 20, 25, 50], \
							'min_samples_leaf_list': [1, 2, 3, 5, 10, 15, 20, 25, 50], \
							'random_state': 42, \
							'folds_cross_val': 5, \
							'scorer_cross_val': 'accuracy', \
							'plot_confusion_matrix': False, \
							'print_scores': True},
		svm_params: dict = {'kernel_list': ['linear', 'poly', 'rbf', 'sigmoid'], \
					  		'C_list': [0.1, 0.5, 1, 2, 5, 10, 20, 50, 100], \
							'gamma_list': ['scale', 'auto'], \
							'scorer_cross_val': 'accuracy', \
							'folds_cross_val': 5, \
							'print_scores': True, \
							'plot_confusion_matrix': False, \
							'random_state': 42}
):
	"""
	Realitza un experiment complet amb els paràmetres especificats.
	"""

	assert 0 < test_size < 1, "El paràmetre 'test_size' ha de ser un valor entre 0 i 1."
	assert scaler in {'', 'standard', 'minmax'}, "El paràmetre 'scaler' ha de ser un dels següents: {'', 'standard', 'minmax'}"
	assert outliers_factor > 0, "El paràmetre 'outliers_factor' ha de ser un valor positiu."
	assert imputer_cross_val_folds > 1, "El paràmetre 'imputer_cross_val_folds' ha de ser un valor positiu."
	assert balancer in {'', 'oversample', 'undersample', 'smote', 'smoteenn'}, "El paràmetre 'balancer' ha de ser un dels següents: {'', 'oversample', 'undersample', 'smote', 'smoteenn'}"
	assert predictor in {'knn', 'dt', 'svm'}, "El paràmetre 'predictor' ha de ser un dels següents: {'knn', 'dt', 'svm'}"

	load_dataset(save_to_csv=False)
	initial_preprocessing(data=data, save_to_csv=False)
	if remove_outliers:
		delete_outliers(data=data, factor=outliers_factor, plots=False, save_to_csv=False)
	if remove_last_rows:
		delete_last_rows(data=data, save_to_csv=False)
	split_dataset(data=data, test_size=test_size, random_state=random_state)
	if scaler:
		scale_variables(data=X_train, scaler=scaler, save_to_csv=False)
		scale_variables(data=X_test, scaler=scaler, save_to_csv=False)
	impute_data(data_to_impute=X_test, \
				numerical_imputer='best', \
				categorical_imputer='best', \
				save_to_csv=False, \
				random_state=random_state, \
				folds_cross_val=imputer_cross_val_folds, \
				num_metric=imputer_num_metric, \
				cat_metric=imputer_cat_metric)
	impute_data(data_to_impute=X_train, \
				numerical_imputer='best', \
				categorical_imputer='best', \
				save_to_csv=False, \
				random_state=random_state, \
				folds_cross_val=imputer_cross_val_folds, \
				num_metric=imputer_num_metric, \
				cat_metric=imputer_cat_metric)
	if balancer:
		balance_target_classes(X_data=X_train, y_data=y_train, method=balancer, random_state=random_state)

	if predictor == 'knn':
		prediction_scores = knn_prediction(X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test, **knn_params)
	elif predictor == 'dt':
		prediction_scores = dt_prediction(X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test, **dt_params)
	elif predictor == 'svm':
		prediction_scores = svm_prediction(X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test, **svm_params)
	
	return prediction_scores
	


In [ ]:
def find_best_experiment(executions_per_experiment: int = 10):
	"""
	Prova totes les combinacions de paràmetres i retorna la millor.
	"""
	from itertools import product

	# Paràmetres
	random_state_list = [42, 123, 456, 789]
	remove_outliers_list = [True, False]
	outliers_factor_list = [3, 5, 10]
	remove_last_rows_list = [True, False]
	test_size_list = [0.15, 0.2, 0.25]
	scaler_list = ['', 'standard', 'minmax']
	imputer_cross_val_folds_list = [5, 10]
	imputer_num_metric_list = ['r2']
	imputer_cat_metric_list = ['accuracy', 'f1_micro', 'f1_macro', 'f1_weighted', 'precision_micro', 'precision_macro', 'precision_weighted', 'recall_micro', 'recall_macro', 'recall_weighted']
	balancer_list = ['', 'oversample', 'undersample', 'smote', 'smoteenn']
	predictor_list = ['knn', 'dt', 'svm']

### **BONUS 1: Explainable Boosting Machine (EBM)**

### **BONUS 2: Anàlisi No Supervisat**